# Project 4, Part 2: Robot localization using particle filtering
## Introduction
In the lecture we have learned particle filtering. which can be interpreted as Monte Carlo method for Hidden Markov Models. In this exercise, we learn how to use this method to track a moving robot's position overtime. Basically, we have access to the steering and velocity control inputs. We also have sensors that measures distance to visible landmarks. Then the basic idea is that the population of particles tracks the high-likelihood regions of the robot's position.

Your tasks is to complete the missing code. Make sure that all the functions follow the provided specification, i.e. the output of the function exactly matches the description in the docstring.
Do not add or modify any code outside of the following comment blocks:
```
##########################################################
# YOUR CODE HERE
.....
##########################################################
```
After you fill in all the missing code, restart the kernel and re-run all the cells in the notebook. You are **NOT** allowed to using additional 'import'  statements. If you plagiarise even for a single project task, you won't be eligible for the bonus this semester.


In [1]:
import copy
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from scipy import stats
from IPython.display import HTML
from ipywidgets import interactive
from matplotlib.patches import Circle

# comment these two lines if you don't want multiple output in a cell
# just for the convenience of debugging
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

## Task 1: Constuct particles
Particles can be constructed by uniformly sampling in the 2D space, or by Gaussian sampling in robot's start point. The latter one is more helpful if you know where the robot's start point is. In this notebook, we have no information about robot's start point, so we just randomly sample particles over the 2D space. In this task, you need to sample particles in the given 2D space with no landmarks, namely, all randomly sampled particles is valid.

In [2]:
def uniform_particles_construction(width, height, N):
    """Sample particles in 2D space with no landmarks randomly.

    Args:
        width, height: int, width and height of the area in which the robot moves
        N: int, number of particles
    
    Return:
        particles: np.ndarray, the coordinate of particles in 2D space
    """
    np.random.seed(500)
    ###################################################################
    # YOUR CODE HERE
    particles = np.random.uniform([0,0], [width, height], size=(N, 2))

    ###################################################################
    
    return particles

Here we show a example map in which the robot moves:

<img src="./img/Map_Rejection_Sampling.png" alt="Encoder" style="width: 400px;">


## Task 2: Find valid particles
The map above contains many irregulately distributed landmarks shown as blue circles. In this case, not all previous sampled particles are valid since these particles can not located inside the landmarks. In this task you need to find the valid particles, namely, only those particles whose distance to each landmarks' center is larger or equal to the landmarks' corresponding radius is considered as valid.

In [3]:
def find_valid_particles(particles, centers, radius):
    """Given randomly sampled particles, as well as centers and radii of landmarks, decide which particles are valid.
    
    Args:
        particles: the particles we get through random generation in 2D space
        centers: centers of landmarks
        radius: the radius of cicular landmarks 
    Return:
        valid_particles: the particles that locate in the valid region
    """
    valid_particles = []

    ###################################################################
    # YOUR CODE HERE
    for count, particle_coord in enumerate(particles):
        dis = np.linalg.norm(particle_coord-centers, axis=1, keepdims=True)
        if np.all(dis >= radius):
            valid_particles.append(particles[count])    
    ###################################################################
    return np.asarray(valid_particles)

## Motion model
Now we have got the uniformly sampled particles in the 2D space and also select the valid particles. Since we have access to the velocity and angle of the robot's motion, now we can move the remaining valid particles based on how you predict the real system is behaving with different noise level in the motion model. Then we can have a look at the influence of different noise level. Assuming that our sensors returns the data in a time interval of 0.5s.

In [4]:
def predict_particle_pos(particles, v, std, dt=0.4):
    """Predict the motion of next state for each particles given current angles and velocities. Note that the preiction here is noised.
    
    Args:
        particles: the particles we get after rejecting the ones that are not valid
        v： 2d array. Each sample with feature [angle, velocity]
        std: control the noise of the distance between time steps
        dt: time interval, according to sensor model, it should be 0.4 second here
    """
    N = len(particles)
    
    # std can be set as a hyperparameter to decide how noisy is the data, thus can change difficulty
    # add some noise to the distance, the level is control in the parameter dis_noise_args
    delta_dist = (v[1] * dt) + (np.random.randn(N) * std) 
    particles[:, 0] += np.cos(v[0]) * delta_dist
    particles[:, 1] += np.sin(v[0]) * delta_dist

## Update the weights of each particle
Update the weighting of the particles based on the measurement. Each particle has a position and a weight which estimates how well it matches the measurement. Normalizing the weights so they sum to one. This normalization step turns them into a probability distribution. Those particles that are closer to the robot will generally have a higher weight than ones far from the robot. Particles that closely match the measurements are weighted higher than particles which don't match the measurements very well. So in this case, we can measure the probability using the distance to landmarks.

In [5]:
def weights_update(particles, weights, dist_r_l, centers, radii, scale_fac, std):
    """Given the noised distances from robot to the landmarks, update the weights of particles
    
    Args:
        particles: coordinate of particles
        weights: weight of particles
        dist_r_l: the current distance between robot and landmarks
        scale_fac, std: hyperparameters to avoid the underflow of possibilities
    """
    
    weights.fill(1.)
    
    for count, center in enumerate(centers):
        # distance between the particles and each landmark
        dist_p_l = np.linalg.norm(particles-center, axis=1, keepdims=True) - radii[count]
        
        weights *= stats.norm.pdf(dist_p_l/scale_fac, dist_r_l[count]/scale_fac, std)

    weights += 1.e-100   # avoid round-off to zero
    weights /= sum(weights)

## Task 3: Resample Procedures
Discard highly impossible particles and replace them with copies of the more possible particles. Accordingly, particles with greater weights survive with higher likelihood than particles with valus of small importance. In principle, there are many ways to achieve this, here you can refer to the procedure given in the paper [Resampling in Particle Filtering - Comparison](http://sait.cie.put.poznan.pl/38/SAIT_38_02.pdf) to complete the systematic resampling method, in which the algorithm is described as below:

<img src="img/systematic_resample.png" alt="Encoder" style="width: 400px;"/>

This resampling has a complexity of *O(N)*, and is one of the more readily recommended, because of its simplicity and operation speed. This approach assumes that the range $[0, 1)$ is subdivided in to N equal parts, and the draw occurs in each stratum
$u^i \sim [\frac{i-1}{N}, \frac{i}{N})$, particles are selected for replication, such that $u^i \in [\sum_p^{j-1}q_p, \sum_p^{j}q_p)$

In [6]:
def resample_procedure(weights):
    """Perform resampling procedure described above
    
    Args:
        weights: the weights to be updated
    
    ReturnL:
        idx: the indices of those remained particled
    """
    
    num_weights = len(weights)
    
    idx = np.zeros(num_weights, 'i') # set the data type as int
    sum_Q = np.cumsum(weights)

    # make N subdivisions, choose positions with a consistent random offset
    U = (np.arange(num_weights) + np.random.uniform()) / num_weights
 
    ###################################################################
    # YOUR CODE HERE
    i, j = 0, 0
    while i<num_weights and j<num_weights:
        if U[i] < sum_Q[j]:
            idx[i] = j
            i += 1
        else:
            j += 1
    ###################################################################
    return idx

The function above takes an array of weights and returns indexes of particles that have been chosen. We just need to write a function that performs the resampling from these indexes:

In [7]:
def resample_from_index(particles, weights, idx):
    particles[:] = particles[idx]
    weights[:] = weights[idx]
    weights /= np.sum(weights)

## Load the data

In [8]:
# Get the information about the maps
landmark_centers = np.load('./data/centers/example_centers.npy')
landmark_radii = np.load('./data/radii/example_radii.npy')
# input the origin data as reference, you should not touch this data in you implementation
input_sequence = np.load('./data/trajectory/example_trajectory.npy')
# Now let's input the velocity and distance data, which should be intepreted
# as corresponding transition and observability matrix in HMM type
angle_velocity = np.load('./data/velocity/example_velocity.npy')
dist_r_l = np.load('./data/distance/example_distance.npy')

## Set parameters

In [9]:
# set hyperparametes
width_max = 800
height_max = 600 
particle_number = 500

# add noise to the distance prediction
dis_noise_args = {'no noise': 0, 'low noise': 1, 'high noise': 5}

# decide the convergence of particles
weight_conv_args = {'scale': 50, 'std':1}

In [10]:
def particle_trajectory_record(width, height, N, centers, radii, velocity, distance, noise, weighting):
    # First we initialize the particles and the weights
    random_particles = uniform_particles_construction(width, height, N)
    remaining_particles = find_valid_particles(random_particles, centers, radii)
    origin_random_weights = np.ones((len(remaining_particles),1))

    # Now we need to record the coordinates of moving particles
    prediction_particles = [remaining_particles]

    random_pos = copy.copy(remaining_particles)
    random_weights = copy.copy(origin_random_weights)

    for i in range(len(distance)):

        # The predicted position of particles
        predict_particle_pos(random_pos, velocity[i], noise, 0.4)
        weights_update(random_pos, random_weights, distance[i], centers, radii, weighting['scale'], weighting['std'])
        random_index = resample_procedure(random_weights)
        resample_from_index(random_pos, random_weights, random_index)
        prediction_particles.append(copy.copy(random_pos))
    
    particle_trajectory = np.asarray(prediction_particles)
    return particle_trajectory, random_weights

In [11]:
trajectory_no_noise, weight_no_noise = particle_trajectory_record(width_max, height_max, particle_number, landmark_centers, landmark_radii, angle_velocity, dist_r_l, dis_noise_args['no noise'], weight_conv_args)

trajectory_low_noise, weight_low_noise = particle_trajectory_record(width_max, height_max, particle_number, landmark_centers, landmark_radii, angle_velocity, dist_r_l, dis_noise_args['low noise'], weight_conv_args)

trajectory_high_noise, weight_high_noise = particle_trajectory_record(width_max, height_max, particle_number, landmark_centers, landmark_radii, angle_velocity, dist_r_l, dis_noise_args['high noise'], weight_conv_args)

## Visulization
Now we visualize the moving of the robot to show how your particle filters works! What we need to achieve here is adding the moving of input sequence as well as particles onto that map.

In [12]:
def location(step):
    img = plt.imread('img/Canvas.png')
    fig,ax = plt.subplots(1, 3, figsize=(15, 5))
    ax_unpack = ax.ravel()
    
    ax_unpack[0].scatter(input_sequence[step,0], input_sequence[step,1], s=15, c='r', marker='x')
    ax_unpack[0].scatter(trajectory_no_noise[step,:,0], trajectory_no_noise[step,:,1],s=1, c='g')

    ax_unpack[1].scatter(input_sequence[step,0], input_sequence[step,1], s=15, c='r', marker='x')
    ax_unpack[1].scatter(trajectory_low_noise[step,:,0], trajectory_low_noise[step,:,1],s=1, c='g')

    ax_unpack[2].scatter(input_sequence[step,0], input_sequence[step,1], s=15, c='r', marker='x')
    ax_unpack[2].scatter(trajectory_high_noise[step,:,0], trajectory_high_noise[step,:,1],s=1, c='g')
    # Now, loop through coord arrays, and create a circle at center
    for i in range(0, 3):
        ax_unpack[i].set_aspect('equal')
        for count, value in enumerate(landmark_centers):
            circ = Circle(value,landmark_radii[count], alpha=0.5)
            ax_unpack[i].add_patch(circ)
        ax_unpack[i].imshow(img)

In [13]:
iplot = interactive(location, step=(0, len(trajectory_no_noise)-1))
iplot

interactive(children=(IntSlider(value=25, description='step', max=50), Output()), _dom_classes=('widget-intera…

Then let's predict the position of robot's final position and make comparison with the true result.

In [14]:
position_no_noise = np.average(trajectory_no_noise[-1], weights=weight_no_noise.flatten(), axis=0)
position_low_noise = np.average(trajectory_low_noise[-1], weights=weight_low_noise.flatten(), axis=0)
position_high_noise = np.average(trajectory_high_noise[-1], weights=weight_high_noise.flatten(), axis=0)
print("The estimate position with no noise is {}".format(position_no_noise))
print("The estimate position with low lowise is {}".format(position_low_noise))
print("The estimate position with high noise is {}".format(position_high_noise))
print("The true final position of robot is {}".format(input_sequence[-1]))

The estimate position with no noise is [618.85110432 480.22242078]
The estimate position with low lowise is [611.03029821 484.30986893]
The estimate position with high noise is [607.02302265 493.04249201]
The true final position of robot is [615.12953368 500.55248619]


## Understanding the resule
We can see that the prediction of final position with no noise is not always the best result. Sometimes the prediction with some noise works better. You can relate it to the the gradient descent method when we want to find the minumum value of a non-convex. For example if we get stuck in some local minimum, performing random walks helps to eacape from this local minima. Same is hold for particle filters with different noise levels. Once the minimum offset for particles with no noise is decided, it will always be there and can never be eliminated. But with different noise level, particles may come to some smaller offset 